# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from decimal import Decimal

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
db_name = 'uk_food'
collection_name = 'establishments' 

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [9]:
establishments = db[collection_name]

In [10]:
# review a document in the establishments collection
pprint(establishments.find_one())


{'AddressLine1': 'Folkestone Harbour',
 'AddressLine2': 'Harbour Approach Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'Pick Up Pintxos',
 'BusinessType': 'Takeaway/sandwich shop',
 'BusinessTypeID': 7844,
 'ChangesByServerID': 0,
 'Distance': 4591.914705329384,
 'FHRSID': 1043701,
 'LocalAuthorityBusinessID': 'PI/000076360',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': '',
 'RatingDate': '2018-04-18T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6668a114cf5c32101a53a3d7'),
 'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043701',
            'rel': 'self'}],
 'meta': {'

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [58]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [59]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [60]:
# Check that the new restaurant was inserted
print("Inserted document ID:", result.inserted_id)

Inserted document ID: 6668ae57723575151295126c


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Define the projection to include only BusinessTypeID and BusinessType fields
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Execute the query
result = db.establishments.find_one(query, projection)

# Print the result
print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
# business_type_mapping = {
#     "Restaurant/Cafe/Canteen": 1,
#     "Retailers - other": 2,
#     # Add more mappings as needed
# }

# # Get the BusinessTypeID for "Restaurant/Cafe/Canteen"
# business_type_id = business_type_mapping.get("Restaurant/Cafe/Canteen")

# # Update the new_restaurant document with the correct BusinessTypeID
# new_restaurant["BusinessTypeID"] = business_type_id

# # Print the updated document
# pprint(new_restaurant)
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": 1}}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
# Confirm that the new restaurant was updated
query = {"BusinessName": "Penang Flavours"}

# Execute the query to find the updated restaurant
updated_restaurant = establishments.find_one(query)

# Print the updated restaurant document
if updated_restaurant:
    print("Updated restaurant:")
    pprint(updated_restaurant)
else:
    print("The updated restaurant was not found in the collection.")

Updated restaurant:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6668a478e584d20b29c06ba2'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [64]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the documents matching the query criteria
document_count = establishments.count_documents(query)

# Print the number of documents
print("Number of documents with LocalAuthorityName as 'Dover':", document_count)

Number of documents with LocalAuthorityName as 'Dover': 0


In [69]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}

# Delete all documents matching the query criteria
delete_result = establishments.delete_many(query)

In [70]:
# Check if any remaining documents include Dover
print("Number of documents deleted:", delete_result.deleted_count)


Number of documents deleted: 0


In [68]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

# Print the remaining document
if remaining_document:
    print("A remaining document in the collection:")
    pprint(remaining_document)
else:
    print("No documents found in the collection.")

A remaining document in the collection:
{'AddressLine1': 'Folkestone Harbour',
 'AddressLine2': 'Harbour Approach Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'Pick Up Pintxos',
 'BusinessType': 'Takeaway/sandwich shop',
 'BusinessTypeID': 7844,
 'ChangesByServerID': 0,
 'Distance': 4591.914705329384,
 'FHRSID': 1043701,
 'LocalAuthorityBusinessID': 'PI/000076360',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': '',
 'RatingDate': '2018-04-18T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6668a114cf5c32101a53a3d7'),
 'geocode': {'latitude': '51.0783519967076', 'longitude': '1.18590330311705'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [72]:
# Change the data type from String to Decimal for longitude and latitude
query = {"geocode.longitude": {"$exists": True}, "geocode.latitude": {"$exists": True}}

# Find documents that need to be updated
documents_to_update = establishments.find(query)

# Iterate over the documents and update longitude and latitude fields
for doc in documents_to_update:
    # Convert longitude and latitude from string to float
    longitude = float(doc['geocode']['longitude'])
    latitude = float(doc['geocode']['latitude'])
    
    # Update the document with the new longitude and latitude values
    establishments.update_one(
        {"_id": doc["_id"]},
        {"$set": {"geocode.longitude": longitude, "geocode.latitude": latitude}}
    )

print("Documents updated successfully.")

Documents updated successfully.


Use `update_many` to convert `RatingValue` to integer numbers.

In [73]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [74]:
# Change the data type from String to Integer for RatingValue
result = establishments.update_many(
    {"RatingValue": {"$exists": True}},  # Select documents where RatingValue exists
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  # Use $toInt to convert RatingValue to integer
)

print("Number of documents updated:", result.modified_count)

Number of documents updated: 34694


In [75]:
# Check that the coordinates and rating value are now numbers
sample_documents = establishments.find().limit(5)

# Iterate over the sample documents and print their coordinates and RatingValue
for doc in sample_documents:
    print("Coordinates (latitude, longitude):", doc["geocode"]["latitude"], ",", doc["geocode"]["longitude"])
    print("RatingValue:", doc["RatingValue"])
    print("-" * 50)  # Separator between documents

Coordinates (latitude, longitude): 51.0783519967076 , 1.18590330311705
RatingValue: 5
--------------------------------------------------
Coordinates (latitude, longitude): 51.0783519967076 , 1.18590330311705
RatingValue: 4
--------------------------------------------------
Coordinates (latitude, longitude): 51.0783519967076 , 1.18590330311705
RatingValue: 5
--------------------------------------------------
Coordinates (latitude, longitude): 51.0783519967076 , 1.18590330311705
RatingValue: 5
--------------------------------------------------
Coordinates (latitude, longitude): 51.0783519967076 , 1.18590330311705
RatingValue: 5
--------------------------------------------------
